In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import datetime

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [3]:
pd.reset_option("display.max_columns")
pd.reset_option("display.max_rows")

# **Obtención de la data**

Tenemos dos fuentes de datos diferentes:

**Dataset obtenido en Kaggle:**
    Fuente: https://www.kaggle.com/datasets/eoinamoore/historical-nba-data-and-player-box-scores?select=PlayerStatistics.csv

Este Dataset cuenta con un total de 6 archivos, para efectos de este proyecto nos centraremos en tres de ellos:

1. PlayerStatistics.csv = Este archivo contiene los cuadros de resultados detallados de cada jugador en la historia de la NBA, partido por partido.
2. TeamStatistics.csv = Contiene un registro completo de los cuadros de resultados a nivel de equipo para cada partido jugado en la historia de la NBA.
3. Games.csv = Registros completos de todos los partidos de la NBA, incluidos los equipos locales y visitantes, los resultados de los partidos.
4. LeagueSchedule24_25.csv = El calendario completo de la temporada 2024-2025 de la NBA, incluidas las ubicaciones y los horarios de los partidos.
5. Players.csv = Información biográfica de cada jugador, incluida la altura, el peso y la posición.
6. TeamHistories.csv = Contiene los cambios de nombre, las reubicaciones y las expansiones de los equipos a lo largo de la historia de la liga.

Al ver en detalle el contenido de todas las tablas, decidimos trabajar solo con 3 de ellas:
1. PlayerStatistics.csv
2. TeamStatistics.csv
3. Players.csv

Porque consideramos que son las 3 que contienen la informacion relevante para nuestro proyecto: TeamStatistics, tiene toda la informacion desde la perspectiva de equipos. PlayerStatistics y Players que contienen toda la informacion detallada de los jugadores. Las 3 tablas estan relacionadas mediante alguno de estos ID: ID equipo, ID jugador y ID partido.

**La segunda fuente de datos la obtendremos mediante Web scrapping**

De esta Web sacaremos la información de los salarios dividida en dos dataframes, el de salarios por equipos y el salario por jugadores, para complementar la información del Dataset de Kaggle.

https://hoopshype.com/salaries/

**Importamos todos los CSV**

In [7]:
#Estos archivos los descargamos y leemos a nivel local, por lo que hay que actualizar la ruta segun tu dispositivo
TeamStatistics = pd.read_csv('TeamStatisticsDescargaEnero.csv', low_memory= False)
PlayerStatistics = pd.read_csv('PlayerStatisticsDescargaEnero.csv')
Players = pd.read_csv('Players.csv')

#El archivo TeamsStatistics arroja este warning porque la columna a la que hace referencia tiene datos 1 y 0, pero hay dos filas que tienen valores nulos, que serán eliminados en la limpieza.

**Hacemos el Scrapping para obtener los dataframes de salarios**

In [9]:
nba_salario_2024_2025 = pd.read_html('https://hoopshype.com/salaries/')
nba_salario_2023_2024 = pd.read_html('https://hoopshype.com/salaries/2023-2024')
nba_salario_2022_2023 = pd.read_html('https://hoopshype.com/salaries/2022-2023')
nba_salario_2021_2022 = pd.read_html('https://hoopshype.com/salaries/2021-2022')
nba_salario_2020_2021 = pd.read_html('https://hoopshype.com/salaries/2020-2021')
nba_salario_2019_2020 = pd.read_html('https://hoopshype.com/salaries/2019-2020')
nba_salario_2018_2019 = pd.read_html('https://hoopshype.com/salaries/2018-2019')
nba_salario_2017_2018 = pd.read_html('https://hoopshype.com/salaries/2017-2018')
nba_salario_2016_2017 = pd.read_html('https://hoopshype.com/salaries/2016-2017')
nba_salario_2015_2016 = pd.read_html('https://hoopshype.com/salaries/2015-2016')
nba_salario_2014_2015 = pd.read_html('https://hoopshype.com/salaries/2014-2015')
nba_salario_2013_2014 = pd.read_html('https://hoopshype.com/salaries/2013-2014')
nba_salario_2012_2013 = pd.read_html('https://hoopshype.com/salaries/2012-2013')
nba_salario_2011_2012 = pd.read_html('https://hoopshype.com/salaries/2011-2012')
nba_salario_2010_2011 = pd.read_html('https://hoopshype.com/salaries/2010-2011')
nba_salario_2009_2010 = pd.read_html('https://hoopshype.com/salaries/2009-2010')
nba_salario_2008_2009 = pd.read_html('https://hoopshype.com/salaries/2008-2009')
nba_salario_2007_2008 = pd.read_html('https://hoopshype.com/salaries/2007-2008')
nba_salario_2006_2007 = pd.read_html('https://hoopshype.com/salaries/2006-2007')
nba_salario_2005_2006 = pd.read_html('https://hoopshype.com/salaries/2005-2006')
nba_salario_2004_2005 = pd.read_html('https://hoopshype.com/salaries/2004-2005')
nba_salario_2003_2004 = pd.read_html('https://hoopshype.com/salaries/2003-2004')
nba_salario_2002_2003 = pd.read_html('https://hoopshype.com/salaries/2002-2003')
nba_salario_2001_2002 = pd.read_html('https://hoopshype.com/salaries/2001-2002')
nba_salario_2000_2001 = pd.read_html('https://hoopshype.com/salaries/2000-2001')

In [10]:
# URL para la temporada 2024-2025
url_2024_2025 = 'https://hoopshype.com/salaries/'
response_2024_2025 = requests.get(url_2024_2025)
soup_2024_2025 = BeautifulSoup(response_2024_2025.content, 'html.parser')

# Buscar la tabla
table_2024_2025 = soup_2024_2025.find('table')

# Extraer filas y encabezados
rows_2024_2025 = []
for tr in table_2024_2025.find_all('tr'):
    cells = [td.text.strip() for td in tr.find_all(['th', 'td'])]
    rows_2024_2025.append(cells)

# Crear el DataFrame
df_nba_salario_2024_2025 = pd.DataFrame(rows_2024_2025[1:], columns=rows_2024_2025[0])
df_nba_salario_2024_2025 = df_nba_salario_2024_2025.iloc[:, :-5]

# Mostrar las primeras filas
print(df_nba_salario_2024_2025.head())

            Team       2024/25
0  1.    Phoenix  $220,708,856
1  2.  Minnesota  $204,304,153
2  3.     Boston  $196,051,963
3  4.   New York  $192,251,829
4  5.  LA Lakers  $191,906,169


Transformamos en Data Frame

In [12]:
##transformacion en dataframes, pongo "0", porque cuando le haces un lend, muestra un solo objeto el cual es la tabla
df_nba_salario_2023_2024 =nba_salario_2023_2024[0]
df_nba_salario_2022_2023 =nba_salario_2022_2023[0]
df_nba_salario_2021_2022=nba_salario_2021_2022[0]
df_nba_salario_2020_2021=nba_salario_2020_2021[0]
df_nba_salario_2019_2020=nba_salario_2019_2020[0]
df_nba_salario_2018_2019=nba_salario_2018_2019[0]
df_nba_salario_2017_2018=nba_salario_2017_2018[0]
df_nba_salario_2016_2017=nba_salario_2016_2017[0]
df_nba_salario_2015_2016=nba_salario_2015_2016[0]
df_nba_salario_2014_2015=nba_salario_2014_2015[0]
df_nba_salario_2013_2014=nba_salario_2013_2014[0]
df_nba_salario_2012_2013=nba_salario_2012_2013[0]
df_nba_salario_2011_2012=nba_salario_2011_2012[0]
df_nba_salario_2010_2011=nba_salario_2010_2011[0]
df_nba_salario_2009_2010=nba_salario_2009_2010[0]
df_nba_salario_2008_2009=nba_salario_2008_2009[0]
df_nba_salario_2007_2008=nba_salario_2007_2008[0]
df_nba_salario_2006_2007=nba_salario_2006_2007[0]
df_nba_salario_2005_2006=nba_salario_2005_2006[0]
df_nba_salario_2004_2005=nba_salario_2004_2005[0]
df_nba_salario_2003_2004=nba_salario_2003_2004[0]
df_nba_salario_2002_2003=nba_salario_2002_2003[0]
df_nba_salario_2001_2002=nba_salario_2001_2002[0]
df_nba_salario_2000_2001=nba_salario_2000_2001[0]

In [13]:
dfNbaSalariesTeams = [
    df_nba_salario_2024_2025 ,
    df_nba_salario_2023_2024,
    df_nba_salario_2022_2023,
    df_nba_salario_2021_2022,
    df_nba_salario_2020_2021,
    df_nba_salario_2019_2020,
    df_nba_salario_2018_2019,
    df_nba_salario_2017_2018,
    df_nba_salario_2016_2017,
    df_nba_salario_2015_2016,
    df_nba_salario_2014_2015,
    df_nba_salario_2013_2014,
    df_nba_salario_2012_2013,
    df_nba_salario_2011_2012,
    df_nba_salario_2010_2011,
    df_nba_salario_2009_2010,
    df_nba_salario_2008_2009,
    df_nba_salario_2007_2008,
    df_nba_salario_2006_2007,
    df_nba_salario_2005_2006,
    df_nba_salario_2004_2005,
    df_nba_salario_2003_2004,
    df_nba_salario_2002_2003,
    df_nba_salario_2001_2002,
    df_nba_salario_2000_2001]
dfNbaSalariesTeams = [df.iloc[:, 1:] for df in dfNbaSalariesTeams]

In [14]:
#En la web hacen ajuste de salarios teniendo en cuenta la inflación, por lo tanto nos quedamos con el valor ajustado:
FilteredDataframeTeamSalaries = []

for df in dfNbaSalariesTeams:
    if not df.empty:
        # Mantener siempre la columna 'Team' si existe
        columns_to_keep0 = ['Team'] if 'Team' in df.columns else []

        # Filtrar las columnas con asterisco
        columns_with_asterisk0 = [col for col in df.columns if '*' in col]

        # Combinar columnas relevantes
        columns_to_keep0.extend(columns_with_asterisk0)

        # Filtrar el DataFrame solo con las columnas seleccionadas
        if columns_to_keep0:
            FilteredDataframeTeamSalaries.append(df[columns_to_keep0])

# Mostrar el resultado del primer DataFrame filtrado
if FilteredDataframeTeamSalaries:
    print(FilteredDataframeTeamSalaries)

[             Team
0         Phoenix
1       Minnesota
2          Boston
3        New York
4       LA Lakers
5          Denver
6       Milwaukee
7          Dallas
8    Golden State
9           Miami
10    New Orleans
11    LA Clippers
12        Toronto
13     Sacramento
14      Cleveland
15   Philadelphia
16     Washington
17       Brooklyn
18        Houston
19        Atlanta
20        Indiana
21       Portland
22        Chicago
23  Oklahoma City
24        Memphis
25    San Antonio
26      Charlotte
27           Utah
28        Orlando
29        Detroit,              Team    2023/24(*)
0    Golden State  $215,575,491
1     LA Clippers  $207,350,074
2         Phoenix  $199,598,596
3       Milwaukee  $192,913,480
4          Boston  $192,495,669
5          Denver  $186,298,923
6           Miami  $182,407,172
7       LA Lakers  $174,924,626
8          Dallas  $172,740,566
9     New Orleans  $172,378,150
10      Cleveland  $171,832,777
11      Minnesota  $171,379,743
12   Philadelphia  $171,

In [15]:
# Desempaquetar la lista de filtered_dataframes
Dataframes0 = [df_nba_salario_2024_2025] + FilteredDataframeTeamSalaries

In [16]:
# Realizar el merge sucesivamente, ya que tenemos los df
mergedDfTeams = Dataframes0[0]
for df2 in Dataframes0[1:]:
    mergedDfTeams = pd.merge(mergedDfTeams, df2, on='Team', how='left')
mergedDfTeams = mergedDfTeams.drop(mergedDfTeams.columns[0], axis=1)

mergedDfTeams

,Team,2024/25,2023/24(*),2022/23(*),2021/22(*),2020/21(*),2019/20(*),2018/19(*),2017/18(*),2016/17(*),...,2009/10(*),2008/09(*),2007/08(*),2006/07(*),2005/06(*),2004/05(*),2003/04(*),2002/03(*),2001/02(*),2000/01(*)
0,Phoenix,"$220,708,856","$199,598,596","$186,655,694","$157,814,221","$157,038,429","$120,864,909","$135,516,107","$118,874,973","$116,998,090",...,"$95,098,250","$109,308,179","$107,548,369","$101,949,887","$86,623,329","$73,862,884","$114,470,809","$96,444,346","$99,894,176","$98,236,801"
1,Minnesota,"$204,304,153","$171,379,743","$154,583,256","$158,919,956","$158,838,062","$140,076,911","$152,060,122","$148,875,924","$108,880,975",...,"$90,677,248","$95,402,059","$104,111,617","$103,466,650","$99,874,021","$116,136,117","$121,893,806","$103,942,847","$97,242,068","$86,541,172"
2,Boston,"$196,051,963","$192,495,669","$189,402,747","$157,908,082","$162,002,554","$144,438,990","$156,265,226","$147,862,224","$121,835,160",...,"$121,700,763","$113,699,674","$112,353,166","$97,521,952","$91,922,105","$106,950,918","$102,933,630","$91,769,470","$83,866,497","$93,812,764"
3,New York,"$192,251,829","$169,893,032","$157,970,114","$139,506,473","$124,473,662","$122,940,807","$153,837,075","$135,188,201","$133,733,931",...,"$124,843,928","$138,761,126","$145,045,534","$181,708,756","$204,546,649","$169,869,031","$152,974,005","$163,203,998","$151,780,123","$134,868,788"
4,LA Lakers,"$191,906,169","$174,924,626","$179,603,738","$190,114,264","$169,806,014","$152,058,818","$133,686,642","$132,268,307","$123,551,292",...,"$133,099,830","$112,345,455","$109,514,294","$119,383,317","$117,784,029","$107,748,764","$109,667,222","$110,286,387","$95,773,977","$107,461,347"
5,Denver,"$185,864,258","$186,298,923","$172,125,732","$159,534,234","$158,177,865","$157,915,026","$147,527,819","$137,933,530","$109,283,609",...,"$109,149,909","$100,679,866","$125,093,110","$101,449,214","$91,251,553","$76,352,236","$67,381,969","$85,763,667","$96,124,258","$95,107,581"
6,Milwaukee,"$185,323,374","$192,913,480","$193,959,293","$186,027,881","$166,502,403","$150,391,306","$163,314,041","$154,578,435","$125,459,700",...,"$97,264,272","$102,459,760","$94,351,897","$98,322,546","$101,799,984","$95,143,914","$91,036,902","$101,281,964","$99,237,401","$105,367,649"
7,Dallas,"$178,518,805","$172,740,566","$187,929,929","$146,505,707","$155,575,488","$148,255,740","$108,418,636","$109,584,150","$133,423,098",...,"$129,513,735","$136,466,588","$153,346,197","$137,084,724","$159,033,650","$152,215,779","$136,028,569","$127,382,002","$102,301,160","$95,263,612"
8,Golden State,"$177,010,942","$215,575,491","$203,984,702","$206,963,957","$208,524,600","$158,539,039","$182,393,116","$176,496,341","$132,419,209",...,"$95,191,218","$96,796,633","$93,713,720","$100,666,627","$92,256,901","$90,996,010","$91,288,107","$83,368,514","$84,158,566","$77,200,694"
9,Miami,"$176,102,077","$182,407,172","$160,536,360","$162,860,249","$164,195,796","$159,290,850","$190,971,244","$171,384,276","$131,318,946",...,"$108,090,206","$100,932,646","$112,742,426","$99,743,680","$98,100,857","$98,580,937","$80,979,792","$100,204,748","$94,166,335","$133,893,083"


In [17]:
mergedDfTeams.columns = mergedDfTeams.columns.str.replace('*', '', regex=False)  # Eliminar asteriscos
mergedDfTeams.columns = mergedDfTeams.columns.str.replace('(', '', regex=False)  # Eliminar paréntesis (
mergedDfTeams.columns = mergedDfTeams.columns.str.replace(')', '', regex=False)  # Eliminar paréntesis )
mergedDfTeams = mergedDfTeams.replace(',', '', regex=True)  # Eliminar las comas de los numeros
# Mostrar el resultado final
mergedDfTeams

,Team,2024/25,2023/24,2022/23,2021/22,2020/21,2019/20,2018/19,2017/18,2016/17,...,2009/10,2008/09,2007/08,2006/07,2005/06,2004/05,2003/04,2002/03,2001/02,2000/01
0,Phoenix,$220708856,$199598596,$186655694,$157814221,$157038429,$120864909,$135516107,$118874973,$116998090,...,$95098250,$109308179,$107548369,$101949887,$86623329,$73862884,$114470809,$96444346,$99894176,$98236801
1,Minnesota,$204304153,$171379743,$154583256,$158919956,$158838062,$140076911,$152060122,$148875924,$108880975,...,$90677248,$95402059,$104111617,$103466650,$99874021,$116136117,$121893806,$103942847,$97242068,$86541172
2,Boston,$196051963,$192495669,$189402747,$157908082,$162002554,$144438990,$156265226,$147862224,$121835160,...,$121700763,$113699674,$112353166,$97521952,$91922105,$106950918,$102933630,$91769470,$83866497,$93812764
3,New York,$192251829,$169893032,$157970114,$139506473,$124473662,$122940807,$153837075,$135188201,$133733931,...,$124843928,$138761126,$145045534,$181708756,$204546649,$169869031,$152974005,$163203998,$151780123,$134868788
4,LA Lakers,$191906169,$174924626,$179603738,$190114264,$169806014,$152058818,$133686642,$132268307,$123551292,...,$133099830,$112345455,$109514294,$119383317,$117784029,$107748764,$109667222,$110286387,$95773977,$107461347
5,Denver,$185864258,$186298923,$172125732,$159534234,$158177865,$157915026,$147527819,$137933530,$109283609,...,$109149909,$100679866,$125093110,$101449214,$91251553,$76352236,$67381969,$85763667,$96124258,$95107581
6,Milwaukee,$185323374,$192913480,$193959293,$186027881,$166502403,$150391306,$163314041,$154578435,$125459700,...,$97264272,$102459760,$94351897,$98322546,$101799984,$95143914,$91036902,$101281964,$99237401,$105367649
7,Dallas,$178518805,$172740566,$187929929,$146505707,$155575488,$148255740,$108418636,$109584150,$133423098,...,$129513735,$136466588,$153346197,$137084724,$159033650,$152215779,$136028569,$127382002,$102301160,$95263612
8,Golden State,$177010942,$215575491,$203984702,$206963957,$208524600,$158539039,$182393116,$176496341,$132419209,...,$95191218,$96796633,$93713720,$100666627,$92256901,$90996010,$91288107,$83368514,$84158566,$77200694
9,Miami,$176102077,$182407172,$160536360,$162860249,$164195796,$159290850,$190971244,$171384276,$131318946,...,$108090206,$100932646,$112742426,$99743680,$98100857,$98580937,$80979792,$100204748,$94166335,$133893083


Inserción de salario por jugador

In [19]:
nba_salario_2024_2025_players= pd.read_html('https://hoopshype.com/salaries/players/')
nba_salario_2023_2024_players= pd.read_html('https://hoopshype.com/salaries/players/2023-2024/')
nba_salario_2022_2023_players= pd.read_html('https://hoopshype.com/salaries/players/2022-2023/')
nba_salario_2021_2022_players= pd.read_html('https://hoopshype.com/salaries/players/2021-2022/')
nba_salario_2020_2021_players= pd.read_html('https://hoopshype.com/salaries/players/2020-2021/')
nba_salario_2019_2020_players= pd.read_html('https://hoopshype.com/salaries/players/2019-2020/')
nba_salario_2018_2019_players= pd.read_html('https://hoopshype.com/salaries/players/2018-2019/')
nba_salario_2017_2018_players= pd.read_html('https://hoopshype.com/salaries/players/2017-2018/')
nba_salario_2016_2017_players= pd.read_html('https://hoopshype.com/salaries/players/2016-2017/')
nba_salario_2015_2016_players= pd.read_html('https://hoopshype.com/salaries/players/2015-2016/')
nba_salario_2014_2015_players = pd.read_html('https://hoopshype.com/salaries/players/2014-2015/')
nba_salario_2013_2014_players = pd.read_html('https://hoopshype.com/salaries/players/2013-2014/')
nba_salario_2012_2013_players = pd.read_html('https://hoopshype.com/salaries/players/2012-2013/')
nba_salario_2011_2012_players = pd.read_html('https://hoopshype.com/salaries/players/2011-2012/')
nba_salario_2010_2011_players = pd.read_html('https://hoopshype.com/salaries/players/2010-2011/')
nba_salario_2009_2010_players = pd.read_html('https://hoopshype.com/salaries/players/2009-2010/')
nba_salario_2008_2009_players = pd.read_html('https://hoopshype.com/salaries/players/2008-2009/')
nba_salario_2007_2008_players = pd.read_html('https://hoopshype.com/salaries/players/2007-2008/')
nba_salario_2006_2007_players = pd.read_html('https://hoopshype.com/salaries/players/2006-2007/')
nba_salario_2005_2006_players = pd.read_html('https://hoopshype.com/salaries/players/2005-2006/')
nba_salario_2004_2005_players = pd.read_html('https://hoopshype.com/salaries/players/2004-2005/')
nba_salario_2003_2004_players = pd.read_html('https://hoopshype.com/salaries/players/2003-2002/')
nba_salario_2002_2003_players = pd.read_html('https://hoopshype.com/salaries/players/2002-2003/')
nba_salario_2001_2002_players= pd.read_html('https://hoopshype.com/salaries/players/2001-2002/')
nba_salario_2000_2001_players= pd.read_html('https://hoopshype.com/salaries/players/2000-2001/')

In [20]:
##transformacion en dataframes, pongo "0", porque cuando le haces un len, muestra un solo objeto el cual es la tabla
df_nba_salario_2024_2025_players =nba_salario_2024_2025_players[0]
df_nba_salario_2023_2024_players =nba_salario_2023_2024_players[0]
df_nba_salario_2022_2023_players =nba_salario_2022_2023_players[0]
df_nba_salario_2021_2022_players=nba_salario_2021_2022_players[0]
df_nba_salario_2020_2021_players=nba_salario_2020_2021_players[0]
df_nba_salario_2019_2020_players=nba_salario_2019_2020_players[0]
df_nba_salario_2018_2019_players=nba_salario_2018_2019_players[0]
df_nba_salario_2017_2018_players=nba_salario_2017_2018_players[0]
df_nba_salario_2016_2017_players=nba_salario_2016_2017_players[0]
df_nba_salario_2015_2016_players=nba_salario_2015_2016_players[0]
df_nba_salario_2014_2015_players=nba_salario_2014_2015_players[0]
df_nba_salario_2013_2014_players=nba_salario_2013_2014_players[0]
df_nba_salario_2012_2013_players=nba_salario_2012_2013_players[0]
df_nba_salario_2011_2012_players=nba_salario_2011_2012_players[0]
df_nba_salario_2010_2011_players=nba_salario_2010_2011_players[0]
df_nba_salario_2009_2010_players=nba_salario_2009_2010_players[0]
df_nba_salario_2008_2009_players=nba_salario_2008_2009_players[0]
df_nba_salario_2007_2008_players=nba_salario_2007_2008_players[0]
df_nba_salario_2006_2007_players=nba_salario_2006_2007_players[0]
df_nba_salario_2005_2006_players=nba_salario_2005_2006_players[0]
df_nba_salario_2004_2005_players=nba_salario_2004_2005_players[0]
df_nba_salario_2003_2004_players=nba_salario_2003_2004_players[0]
df_nba_salario_2002_2003_players=nba_salario_2002_2003_players[0]
df_nba_salario_2001_2002_players=nba_salario_2001_2002_players[0]
df_nba_salario_2000_2001_players=nba_salario_2000_2001_players[0]

In [21]:
#Detectamos que en algunos años, el scrapping no ha funcionado por lo que lo hacemos de la siguiente manera:
url = 'https://hoopshype.com/salaries/players/2020-2021/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Buscar la tabla
table = soup.find('table')

# Extraer filas y encabezados
rows = []
for tr in table.find_all('tr'):
    cells = [td.text.strip() for td in tr.find_all(['th', 'td'])]
    rows.append(cells)

# Crear el DataFrame
df_nba_salario_2020_2021_players = pd.DataFrame(rows[1:], columns=rows[0])  # Salta el encabezado

print(df_nba_salario_2020_2021_players.head())

                  Player      2020/21   2020/21(*)
0  1.      Stephen Curry  $43,006,362  $52,411,485
1  2.         Chris Paul  $41,358,814  $50,403,633
2  2.  Russell Westbrook  $41,358,814  $50,403,633
3  4.       James Harden  $41,254,920  $50,277,018
4  4.          John Wall  $41,254,920  $50,277,018


In [22]:
# URL para la temporada 2016-2017
url_2016_2017 = 'https://hoopshype.com/salaries/players/2016-2017/'
response_2016_2017 = requests.get(url_2016_2017)
soup_2016_2017 = BeautifulSoup(response_2016_2017.content, 'html.parser')

# Buscar la tabla
table_2016_2017 = soup_2016_2017.find('table')

# Extraer filas y encabezados
rows_2016_2017 = []
for tr in table_2016_2017.find_all('tr'):
    cells = [td.text.strip() for td in tr.find_all(['th', 'td'])]
    rows_2016_2017.append(cells)

# Crear el DataFrame
df_nba_salario_2016_2017_players = pd.DataFrame(rows_2016_2017[1:], columns=rows_2016_2017[0])

# Mostrar las primeras filas
print(df_nba_salario_2016_2017_players.head())

             Player      2016/17   2016/17(*)
0  1.  LeBron James  $30,963,450  $40,361,889
1  2.    Al Horford  $26,540,100  $34,595,905
2  2.  Kevin Durant  $26,540,100  $34,595,905
3  2.  James Harden  $26,540,100  $34,595,905
4  2.   Mike Conley  $26,540,100  $34,595,905


In [23]:
# URL para la temporada 2012-2013
url_2012_2013 = 'https://hoopshype.com/salaries/players/2012-2013/'
response_2012_2013 = requests.get(url_2012_2013)
soup_2012_2013 = BeautifulSoup(response_2012_2013.content, 'html.parser')

# Buscar la tabla
table_2012_2013 = soup_2012_2013.find('table')

# Extraer filas y encabezados
rows_2012_2013 = []
for tr in table_2012_2013.find_all('tr'):
    cells = [td.text.strip() for td in tr.find_all(['th', 'td'])]
    rows_2012_2013.append(cells)

# Crear el DataFrame
df_nba_salario_2012_2013_players = pd.DataFrame(rows_2012_2013[1:], columns=rows_2012_2013[0])

# Mostrar las primeras filas
print(df_nba_salario_2012_2013_players.head())

                 Player      2012/13   2012/13(*)
0  1.       Kobe Bryant  $27,849,000  $38,127,661
1  2.       Brandon Roy  $21,459,805  $29,380,306
2  3.     Dirk Nowitzki  $20,907,128  $28,623,645
3  4.    Gilbert Arenas  $20,807,922  $28,487,824
4  5.  Amare Stoudemire  $19,948,799  $27,311,611


In [24]:
##primero tengo que borrar las dos ultimas columnas de el primer df, ya que hacen una peuqeña prediccion, o muestra parte de un contrato ya firmado futuro
df_nba_salario_2024_2025_players = df_nba_salario_2024_2025_players.iloc[:, :-5]

In [25]:
# Crear una lista de DataFrames (uno por cada año)
dfsSalarioPlayers = [
    df_nba_salario_2024_2025_players ,
    df_nba_salario_2023_2024_players,
    df_nba_salario_2022_2023_players ,
    df_nba_salario_2021_2022_players,
    df_nba_salario_2020_2021_players,
    df_nba_salario_2019_2020_players,
    df_nba_salario_2018_2019_players,
    df_nba_salario_2017_2018_players,
    df_nba_salario_2016_2017_players,
    df_nba_salario_2015_2016_players,
    df_nba_salario_2014_2015_players,
    df_nba_salario_2013_2014_players,
    df_nba_salario_2012_2013_players,
    df_nba_salario_2011_2012_players,
    df_nba_salario_2010_2011_players,
    df_nba_salario_2009_2010_players,
    df_nba_salario_2008_2009_players,
    df_nba_salario_2007_2008_players,
    df_nba_salario_2006_2007_players,
    df_nba_salario_2005_2006_players,
    df_nba_salario_2004_2005_players,
    df_nba_salario_2003_2004_players,
    df_nba_salario_2002_2003_players,
    df_nba_salario_2001_2002_players,
    df_nba_salario_2000_2001_players]
dfsSalarioPlayers = [df.iloc[:, 1:] for df in dfsSalarioPlayers]

In [26]:
#En la web hacen ajuste de salarios teniendo en cuenta la inflación, por lo tanto nos quedamos con el valor ajustado:
FilteredDataframes = []

for df in dfsSalarioPlayers:
    if not df.empty:
        # Mantener siempre la columna 'Player' si existe
        columns_to_keep = ['Player'] if 'Player' in df.columns else []

        # Filtrar las columnas con asterisco
        columns_with_asterisk = [col for col in df.columns if '*' in col]

        # Combinar columnas relevantes
        columns_to_keep.extend(columns_with_asterisk)

        # Filtrar el DataFrame solo con las columnas seleccionadas
        if columns_to_keep:
            FilteredDataframes.append(df[columns_to_keep])

# Mostrar el resultado del primer DataFrame filtrado
if FilteredDataframes:
    print(FilteredDataframes)

[               Player
0       Stephen Curry
1        Nikola Jokic
2         Joel Embiid
3        Kevin Durant
4        Bradley Beal
..                ...
575        Alex Reese
576   Jaylin Galloway
577  Anzejs Pasecniks
578    Taevion Kinsey
579      Malcolm Hill

[580 rows x 1 columns],                  Player   2023/24(*)
0         Stephen Curry  $53,458,234
1          Kevin Durant  $49,065,286
2          Nikola Jokic  $49,021,953
3          LeBron James  $49,021,953
4           Joel Embiid  $49,021,953
..                  ...          ...
607      Jalen Crutcher      $66,254
608  Izaiah Brockington      $66,254
609      Taevion Kinsey      $66,254
610       Edmond Sumner      $41,661
611        Kaiser Gates      $36,440

[612 rows x 2 columns],                 Player   2022/23(*)
0        Stephen Curry  $50,968,059
1            John Wall  $50,200,141
2    Russell Westbrook  $49,918,548
3         LeBron James  $47,156,296
4         Kevin Durant  $46,779,742
..                 ...   

In [27]:
# Desempaquetar la lista de filtered_dataframes
Dataframes1 = [df_nba_salario_2024_2025_players] + FilteredDataframes


In [28]:
# Realizar el merge sucesivamente, ya que tenemos los df
MergedDfPlayers = Dataframes1[0]
for df in Dataframes1[1:]:
    MergedDfPlayers = pd.merge(MergedDfPlayers, df, on='Player', how='left')
# Eliminar la primera columna usando iloc
MergedDfPlayers = MergedDfPlayers.iloc[:, 1:]

In [29]:
# Eliminar los asteriscos y los paréntesis de los nombres de las columnas en merged_df_players
MergedDfPlayers.columns = MergedDfPlayers.columns.str.replace('*', '', regex=False)  # Eliminar asteriscos
MergedDfPlayers.columns = MergedDfPlayers.columns.str.replace('(', '', regex=False)  # Eliminar paréntesis (
MergedDfPlayers.columns = MergedDfPlayers.columns.str.replace(')', '', regex=False)  # Eliminar paréntesis )
MergedDfPlayers = MergedDfPlayers.replace(',', '', regex=True)  # Eliminar las comas de los numeros
# Ver el DataFrame con los nombres de columnas actualizados
MergedDfPlayers

C:\Users\andre\AppData\Local\Temp\ipykernel_29804\61584708.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  MergedDfPlayers = MergedDfPlayers.replace(',', '', regex=True)  # Eliminar las comas de los numeros


,Player,2024/25,2023/24,2022/23,2021/22,2020/21,2019/20,2018/19,2017/18,2016/17,...,2010/11,2009/10,2008/09,2007/08,2006/07,2005/06,2004/05,2002/03,2001/02,2000/01
0,Stephen Curry,$55761217,$53458234,$50968059,$52938707,$52411485,$49346703,$46700853,$44483232,$15788863,...,$4200012,$3948158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nikola Jokic,$51415938,$49021953,$35040189,$35898700,$36002595,$33736104,$31752073,$1887168,$1770850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Joel Embiid,$51415938,$49021953,$35643458,$36516749,$36002595,$33736104,$31752073,$7824094,$6291060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kevin Durant,$51179020,$49065286,$46779742,$48588451,$48880434,$45626840,$37403418,$32064562,$34595905,...,$8725756,$6987059,$6438193,$6289772,NaN,NaN,NaN,NaN,NaN,NaN
4,Bradley Beal,$50203930,$48130468,$45888469,$38996895,$35039541,$33231237,$31710944,$30494048,$28829921,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,Alex Reese,$79804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
576,Jaylin Galloway,$77500,$145760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
577,Anzejs Pasecniks,$77500,NaN,NaN,NaN,$456152,$591379,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
578,Taevion Kinsey,$77500,$66254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Limpieza TeamStatistics y PlayerStatistics

In [31]:
TeamStatistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129114 entries, 0 to 129113
Data columns (total 48 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   gameId                   129114 non-null  int64  
 1   gameDate                 129114 non-null  object 
 2   teamCity                 129114 non-null  object 
 3   teamName                 129114 non-null  object 
 4   teamId                   129114 non-null  int64  
 5   opponentTeamCity         129114 non-null  object 
 6   opponentTeamName         129114 non-null  object 
 7   opponentTeamId           129114 non-null  int64  
 8   home                     129114 non-null  bool   
 9   win                      129112 non-null  object 
 10  teamScore                129114 non-null  int64  
 11  opponentScore            129114 non-null  int64  
 12  assists                  129112 non-null  float64
 13  blocks                   129112 non-null  float64
 14  stea

Separamos la columna de GameDate en fecha y hora para poder convertir en formato Fecha y filtrar el dataframe, ya que queramos quedarnos solo con la data del 2000 en adelante

In [33]:
TeamStatistics[['Date', 'Time']] = TeamStatistics['gameDate'].str.split(' ', expand=True)
TeamStatistics['Date'] = pd.to_datetime(TeamStatistics['Date'])
TeamStatistics = TeamStatistics[TeamStatistics['Date'].dt.year >= 2000]
TeamStatistics = TeamStatistics.drop(columns=['Date', 'Time'])
TeamStatistics['gameDate'] = pd.to_datetime(TeamStatistics['gameDate'])

In [34]:
PlayerStatistics[['Date', 'Time']] = PlayerStatistics['gameDate'].str.split(' ', expand=True)
PlayerStatistics['Date'] = pd.to_datetime(PlayerStatistics['Date'])
PlayerStatistics = PlayerStatistics[PlayerStatistics['Date'].dt.year >= 2000]
PlayerStatistics = PlayerStatistics.drop(columns=['Date', 'Time'])
PlayerStatistics['gameDate'] = pd.to_datetime(PlayerStatistics['gameDate'])

El warning que arroja la columna WIN al inicio es porque este partido fue aplazado y por eso tiene como nulos los datos de esas dos filas

In [36]:
RowsWithNulls = TeamStatistics[TeamStatistics['win'].isnull()]
RowsWithNulls

,gameId,gameDate,teamCity,teamName,teamId,opponentTeamCity,opponentTeamName,opponentTeamId,home,win,...,leadChanges,pointsFastBreak,pointsFromTurnovers,pointsInThePaint,pointsSecondChance,timesTied,timeoutsRemaining,seasonWins,seasonLosses,coachId
134,22400524,2025-01-09 22:30:00,Los Angeles,Lakers,1610612747,Charlotte,Hornets,1610612766,True,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,16.0,NaN
135,22400524,2025-01-09 22:30:00,Charlotte,Hornets,1610612766,Los Angeles,Lakers,1610612747,False,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,27.0,NaN


In [37]:
#Como ese partido no se llevó a cabo eliminamos las dos filas, para quitar esta data que no aporta valor porque este partido no se hizo
TeamStatistics.dropna(subset=['win'], inplace=True)

Verificamos que la columna coachID solo tiene valores nulos por lo tanto la elimnamos también

In [39]:
TeamStatistics['coachId'].unique()

array([nan])

In [40]:
#Eliminamos la columna coachId
TeamStatistics.drop(['coachId'], axis=1, inplace=True)

In [41]:
#Convertimos los valores de win y home en enteros para que estén igual al dataframe de PlayersStatistics
TeamStatistics['win'] = TeamStatistics['win'].astype(int)
TeamStatistics['home'] = TeamStatistics['home'].astype(int)

In [42]:
TeamStatistics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62280 entries, 0 to 62281
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   gameId                   62280 non-null  int64         
 1   gameDate                 62280 non-null  datetime64[ns]
 2   teamCity                 62280 non-null  object        
 3   teamName                 62280 non-null  object        
 4   teamId                   62280 non-null  int64         
 5   opponentTeamCity         62280 non-null  object        
 6   opponentTeamName         62280 non-null  object        
 7   opponentTeamId           62280 non-null  int64         
 8   home                     62280 non-null  int32         
 9   win                      62280 non-null  int32         
 10  teamScore                62280 non-null  int64         
 11  opponentScore            62280 non-null  int64         
 12  assists                  62280 non-nu

Las ultimas 12 columnas solo tienen 686 valores, por lo que las eliminamos ya que no tienen suficiente data para que sea significativo quedarnos con ellas

In [44]:
TeamStatistics = TeamStatistics.iloc[:, :-12]

In [45]:
TeamStatistics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62280 entries, 0 to 62281
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   gameId                   62280 non-null  int64         
 1   gameDate                 62280 non-null  datetime64[ns]
 2   teamCity                 62280 non-null  object        
 3   teamName                 62280 non-null  object        
 4   teamId                   62280 non-null  int64         
 5   opponentTeamCity         62280 non-null  object        
 6   opponentTeamName         62280 non-null  object        
 7   opponentTeamId           62280 non-null  int64         
 8   home                     62280 non-null  int32         
 9   win                      62280 non-null  int32         
 10  teamScore                62280 non-null  int64         
 11  opponentScore            62280 non-null  int64         
 12  assists                  62280 non-nu

Durante los años 2003 y 2013 los Hornets pasaron a llamarse Bobcats, para que el equipo tenga el mismo nombre en toda la data y sea mas sencillo manipular el dato, reemplazamos el valor en ambos dataframes

In [47]:
TeamStatistics['teamName'] = TeamStatistics['teamName'].replace('Bobcats', 'Hornets')
TeamStatistics['opponentTeamName'] = TeamStatistics['opponentTeamName'].replace('Bobcats', 'Hornets')
TeamStatistics['teamName'] = TeamStatistics['teamName'].replace('SuperSonics', 'Thunder')
TeamStatistics['opponentTeamName'] = TeamStatistics['opponentTeamName'].replace('SuperSonics', 'Thunder')
PlayerStatistics['playerteamName'] = PlayerStatistics['playerteamName'].replace('Bobcats', 'Hornets')
PlayerStatistics['opponentteamName'] = PlayerStatistics['opponentteamName'].replace('Bobcats', 'Hornets')
PlayerStatistics['playerteamName'] = PlayerStatistics['playerteamName'].replace('SuperSonics', 'Thunder')
PlayerStatistics['opponentteamName'] = PlayerStatistics['opponentteamName'].replace('SuperSonics', 'Thunder')

Verificamos que ambos dataframes tengan la misma cantidad de valores unos para partidos:

In [49]:
Unique1 = TeamStatistics['gameId'].nunique()
Unique2 = PlayerStatistics['gameId'].nunique()

print(f"Cantidad de valores únicos en df1['ID']: {Unique1}")
print(f"Cantidad de valores únicos en df2['ID']: {Unique2}")

Cantidad de valores únicos en df1['ID']: 30925
Cantidad de valores únicos en df2['ID']: 30925


Unimos la columna First y Last Name en una nueva columna llamada Player, para poder relacionar con los salarios, ya que en MergedDfPlayers esta el nombre completo

In [51]:
PlayerStatistics['Player'] = PlayerStatistics['firstName'].str.cat(PlayerStatistics['lastName'], sep=' ')

Verificamos que los valores unicos de la columna Player coincidan en PlayerStatistics y MergedDfPlayers para saber que tenemos los salarios de todos los jugadores

In [53]:
unique_players_PlayerStatistics = set(PlayerStatistics['Player'].unique())
unique_players_Players_Salaries = set(MergedDfPlayers['Player'].unique())

In [54]:
NotInMergedDfPlayers = unique_players_Players_Salaries - unique_players_PlayerStatistics
NumNotInMergedDfPlayers = len(NotInMergedDfPlayers)

print(f"Número de jugadores solo en df2: {NumNotInMergedDfPlayers}\n")
SortedNames = list(NotInMergedDfPlayers)
sorted(SortedNames)

Número de jugadores solo en df2: 62



['AJ Lawson',
 'Aaron Gordon',
 'Alex Sarr',
 'Andre Jackson Jr',
 'BJ Boston',
 'Boo Buie',
 'Cameron Whitmore',
 'Charlie Brown',
 'Craig Porter',
 'DJ Carton',
 'DaRon Holmes',
 'David Duke',
 'David Jones',
 'Dennis Schroeder',
 'Dereck Lively',
 'Derrick Jones',
 'Devonte Graham',
 'Duane Washington Jr',
 'EJ Liddell',
 'Eric Gordon',
 'Gary Trent Jr',
 'Goga Bitadze',
 'Herb Jones',
 'Izaiah Brockington',
 'JD Davison',
 'Jaime Jaquez',
 'Jaren Jackson Jr',
 'Jaylin Galloway',
 'Jeff Dowtin',
 'Jordan Goodwin',
 'Josh Primo',
 'KJ Martin',
 'Keion Brooks',
 'Kelly Oubre',
 'Kevin McCullar',
 'Kevin Porter',
 'Larry Nance Jr',
 'Marvin Bagley',
 'Michael Porter',
 'Nick Smith',
 'Nicolas Claxton',
 'OG Anunoby',
 'PJ Tucker',
 'PJ Washington',
 'Patrick Baldwin',
 'Patrick Mills',
 'Ricky Council',
 'Robert Williams',
 'Ron Harper Jr',
 'Ron Holland',
 'Rudy Gobert',
 'Santiago Aldama',
 'Scotty Pippen Jr',
 'Sviatoslav Mykhailiuk',
 'TJ McConnell',
 'Tim Hardaway Jr',
 'Trey Jemi

Como tenemos una diferencia de 62 personas que estan en la tabla salarios, pero no aparecen en las estadisticas, verificamos uno a uno la razón, al ser un número reducido de datos, da para hacerlo de manera manual

In [56]:
#Verificamos en MergedDfPlayers cuantas columnas de salario tenemos del jugador, para saber si hay histórico
RowSalarie = MergedDfPlayers.loc[MergedDfPlayers["Player"] == "AJ Lawson"]
print(RowSalarie)

        Player  2024/25   2023/24  2022/23 2021/22 2020/21 2019/20 2018/19  \
531  AJ Lawson  $479642  $1029713  $393824     NaN     NaN     NaN     NaN   

    2017/18 2016/17  ... 2010/11 2009/10 2008/09 2007/08 2006/07 2005/06  \
531     NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   

    2004/05 2002/03 2001/02 2000/01  
531     NaN     NaN     NaN     NaN  

[1 rows x 25 columns]


In [57]:
#insertamos estos comandos para poder ver el dataframe completo y localizar al jugador buscado
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [58]:
#Hacemos un filtro por equipos para ubicar a los jugadores y verlos en orden alfabetico
RowTeam = PlayerStatistics.loc[PlayerStatistics["playerteamName"] == "Raptors"]

GroupedTeam = RowTeam.groupby("Player").agg({
    "points": "sum",  # Suma de goles por jugador
    "playerteamName": lambda x: ", ".join(x.unique())  # Equipos únicos del jugador
}).reset_index()

GroupedTeam

,Player,points,playerteamName
0,A.J. Lawson,9,Raptors
1,Aaron Gray,207,Raptors
2,Aaron Williams,67,Raptors
3,Alan Anderson,199,Raptors
4,Aleksandar Radojevic,7,Raptors
5,Alex Len,25,Raptors
6,Alexis Ajinca,114,Raptors
7,Alfonzo McKinnie,46,Raptors
8,Alize Johnson,3,Raptors
9,Alonzo Mourning,0,Raptors


In [59]:
#Volvemos a poner la extension de los dataframes por defecto
pd.reset_option("display.max_columns")
pd.reset_option("display.max_rows")

Hacemos este proceso con todos los jugadores y generamos los siguientes diccionarios para corregir las discordancias de nombres entre las datas

In [61]:
MergedDfPlayers = MergedDfPlayers.copy()

# Diccionario con los valores a reemplazar
replacements = {
    'AJ Lawson': 'A.J. Lawson',
    'Alex Sarr': 'Alexandre Sarr',
    'Andre Jackson Jr': 'Andre Jackson Jr.',
    'BJ Boston': 'Brandon Boston Jr.',
    'Boo Buie': 'Boo Buie III',
    'Cameron Whitmore': 'Cam Whitmore',
    'Charlie Brown': 'Charlie Brown Jr.',
    'Craig Porter': 'Craig Porter Jr.',
    'DJ Carton': 'D.J. Carton',
    'DaRon Holmes': 'DaRon Holmes II',
    'David Duke': 'David Duke Jr.',
    'David Jones': 'David Jones Garcia',
    'Dennis Schroeder': 'Dennis Schroder',
    'Dereck Lively': 'Dereck Lively II',
    'Derrick Jones': 'Derrick Jones Jr.',
    'Devonte Graham': "Devonte' Graham",
    'Duane Washington Jr': "Duane Washington Jr.",
    'EJ Liddell': "E.J. Liddell",
    'Gary Trent Jr': 'Gary Trent Jr.',
    'Herb Jones': 'Herbert Jones',
    'JD Davison': 'J.D. Davison',
    'Jaime Jaquez': 'Jaime Jaquez Jr.',
    'Jaren Jackson Jr': 'Jaren Jackson Jr.',
    'Jeff Dowtin': 'Jeff Dowtin Jr.',
    'Josh Primo': 'Joshua Primo',
    'KJ Martin': 'Kenyon Martin Jr.',
    'Keion Brooks': 'Keion Brooks Jr.',
    'Kelly Oubre': 'Kelly Oubre Jr.',
    'Kevin McCullar': 'Kevin McCullar Jr.',
    'Kevin Porter': 'Kevin Porter Jr.',
    'Larry Nance Jr': 'Larry Nance Jr.',
    'Marvin Bagley': 'Marvin Bagley III',
    'Michael Porter': 'Michael Porter Jr.',
    'Nick Smith': 'Nick Smith Jr.',
    'OG Anunoby': 'O.G. Anunoby',
    'PJ Tucker': 'P.J. Tucker',
    'PJ Washington': 'P.J. Washington',
    'Patrick Baldwin': 'Patrick Baldwin Jr.',
    'Patrick Mills': 'Patty Mills',
    'Ricky Council': 'Ricky Council IV',
    'Robert Williams': 'Robert Williams III',
    'Ron Harper Jr': 'Ron Harper Jr.',
    'Ron Holland': 'Ronald Holland II',
    'Scotty Pippen Jr': 'Scotty Pippen Jr.',
    'TJ McConnell': 'T.J. McConnell',
    'Tim Hardaway Jr': 'Tim Hardaway Jr.',
    'Trey Jemison': 'Trey Jemison III',
    'Trey Murphy': 'Trey Murphy III',
    'TyTy Washington': 'TyTy Washington Jr.',
    'Vince Williams Jr': 'Vince Williams Jr.',
    'Wendell Carter': 'Wendell Carter Jr.',
    'Wendell Moore': 'Wendell Moore Jr.'
}

# Aplicar reemplazos
MergedDfPlayers['Player'] = MergedDfPlayers['Player'].replace(replacements)

In [62]:
PlayerStatistics = PlayerStatistics.copy()

# Diccionario con los valores a reemplazar
replacements = {
    'Aaron rdon': 'Aaron Gordon',
    'Eric rdon': 'Eric Gordon',
    'ga Bitadze': 'Goga Bitadze',
    'Nic Claxton': 'Nicolas Claxton',
    'Rudy bert': 'Rudy Gobert',
    'Santi Aldama': 'Santiago Aldama',
    'Svi Mykhailiuk': 'Sviatoslav Mykhailiuk'
}

# Aplicar reemplazos
PlayerStatistics['Player'] = PlayerStatistics['Player'].replace(replacements)

Encontramos 3 jugadores que no estaban en las estadisticas, son jugadores que se ficharon por poco tiempo y no quedaron registros de su paso por los equipos, al ser un número tan bajo los eliminamos de la data

In [64]:
#Lista de jugadores a eliminar
players_to_remove = ["Izaiah Brockington", "Jaylin Galloway", "KJ Jones"]

# Filtrar el DataFrame eliminando las filas donde 'Player' esté en la lista
MergedDfPlayers = MergedDfPlayers[~MergedDfPlayers["Player"].isin(players_to_remove)]

# Reiniciar los índices después de la eliminación
MergedDfPlayers = MergedDfPlayers.reset_index(drop=True)
MergedDfPlayers

,Player,2024/25,2023/24,2022/23,2021/22,2020/21,2019/20,2018/19,2017/18,2016/17,...,2010/11,2009/10,2008/09,2007/08,2006/07,2005/06,2004/05,2002/03,2001/02,2000/01
0,Stephen Curry,$55761217,$53458234,$50968059,$52938707,$52411485,$49346703,$46700853,$44483232,$15788863,...,$4200012,$3948158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nikola Jokic,$51415938,$49021953,$35040189,$35898700,$36002595,$33736104,$31752073,$1887168,$1770850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Joel Embiid,$51415938,$49021953,$35643458,$36516749,$36002595,$33736104,$31752073,$7824094,$6291060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kevin Durant,$51179020,$49065286,$46779742,$48588451,$48880434,$45626840,$37403418,$32064562,$34595905,...,$8725756,$6987059,$6438193,$6289772,NaN,NaN,NaN,NaN,NaN,NaN
4,Bradley Beal,$50203930,$48130468,$45888469,$38996895,$35039541,$33231237,$31710944,$30494048,$28829921,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,Javon Freeman-Liberty,$100000,$874560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
574,Alex Reese,$79804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
575,Anzejs Pasecniks,$77500,NaN,NaN,NaN,$456152,$591379,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
576,Taevion Kinsey,$77500,$66254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Al haber igualado todos los nombres de la las dos datas, aprovechamos de añadir el ID que tienes los jugadores en la data de salarios que no viene sin ID

In [66]:
# Crear un Series indexado con Player como índice y personId como valores
idUnique = PlayerStatistics.drop_duplicates(subset='Player', keep='last')

teamIdMap = idUnique.set_index('Player')['personId']

# Usar .map() para agregar la columna TeamID a df1
MergedDfPlayers['personId'] = MergedDfPlayers['Player'].map(teamIdMap)
MergedDfPlayers

,Player,2024/25,2023/24,2022/23,2021/22,2020/21,2019/20,2018/19,2017/18,2016/17,...,2009/10,2008/09,2007/08,2006/07,2005/06,2004/05,2002/03,2001/02,2000/01,personId
0,Stephen Curry,$55761217,$53458234,$50968059,$52938707,$52411485,$49346703,$46700853,$44483232,$15788863,...,$3948158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201939.0
1,Nikola Jokic,$51415938,$49021953,$35040189,$35898700,$36002595,$33736104,$31752073,$1887168,$1770850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203999.0
2,Joel Embiid,$51415938,$49021953,$35643458,$36516749,$36002595,$33736104,$31752073,$7824094,$6291060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203954.0
3,Kevin Durant,$51179020,$49065286,$46779742,$48588451,$48880434,$45626840,$37403418,$32064562,$34595905,...,$6987059,$6438193,$6289772,NaN,NaN,NaN,NaN,NaN,NaN,201142.0
4,Bradley Beal,$50203930,$48130468,$45888469,$38996895,$35039541,$33231237,$31710944,$30494048,$28829921,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203078.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,Javon Freeman-Liberty,$100000,$874560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1631241.0
574,Alex Reese,$79804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1642024.0
575,Anzejs Pasecniks,$77500,NaN,NaN,NaN,$456152,$591379,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1628394.0
576,Taevion Kinsey,$77500,$66254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1641795.0


Ahora igualamos las tablas de TeamStatistics y mergedDfTeams para que también tengan los nombres de los equipos escritos igual

In [68]:
TeamStatistics['teamName'].unique()

array(['Pelicans', 'Jazz', 'Celtics', 'Warriors', 'Cavaliers', 'Suns',
       'Hawks', 'Knicks', 'Timberwolves', 'Grizzlies', 'Rockets',
       'Pistons', 'Mavericks', 'Hornets', 'Kings', 'Wizards', 'Bulls',
       'Trail Blazers', 'Clippers', 'Lakers', 'Nets', 'Thunder', 'Bucks',
       '76ers', 'Nuggets', 'Magic', 'Heat', 'Spurs', 'Pacers', 'Raptors'],
      dtype=object)

In [69]:
mergedDfTeams['Team'].unique()

array(['Phoenix', 'Minnesota', 'Boston', 'New York', 'LA Lakers',
       'Denver', 'Milwaukee', 'Dallas', 'Golden State', 'Miami',
       'New Orleans', 'LA Clippers', 'Toronto', 'Sacramento', 'Cleveland',
       'Philadelphia', 'Washington', 'Brooklyn', 'Houston', 'Atlanta',
       'Indiana', 'Portland', 'Chicago', 'Oklahoma City', 'Memphis',
       'San Antonio', 'Charlotte', 'Utah', 'Orlando', 'Detroit'],
      dtype=object)

In [70]:
#Hacemos un mapping para igualar los nombres de los equipos en todos los dataframes
team_mapping = {
    'Phoenix': 'Suns',
    'Minnesota': 'Timberwolves',
    'Boston': 'Celtics',
    'Milwaukee': 'Bucks',
    'New York': 'Knicks',
    'LA Lakers': 'Lakers',
    'Miami': 'Heat',
    'Denver': 'Nuggets',
    'Philadelphia': '76ers',
    'Dallas': 'Mavericks',
    'Golden State': 'Warriors',
    'New Orleans': 'Pelicans',
    'LA Clippers': 'Clippers',
    'Cleveland': 'Cavaliers',
    'Sacramento': 'Kings',
    'Brooklyn': 'Nets',
    'Indiana': 'Pacers',
    'Atlanta': 'Hawks',
    'Memphis': 'Grizzlies',
    'Portland': 'Trail Blazers',
    'Chicago': 'Bulls',
    'Toronto': 'Raptors',
    'Houston': 'Rockets',
    'Oklahoma City': 'Thunder',
    'Washington': 'Wizards',
    'Charlotte': 'Hornets',
    'San Antonio': 'Spurs',
    'Orlando': 'Magic',
    'Utah': 'Jazz',
    'Detroit': 'Pistons'
}
# Realizar el reemplazo usando .replace() y el diccionario
mergedDfTeams['Team'] = mergedDfTeams['Team'].replace(team_mapping)

Ahora que tenemos todos los nombres igualados, añadimos el ID de equipo a la columna de salarios por equipos

In [72]:
# Crear un Series indexado con teamName como índice y TeamID como valores
IdUnique = TeamStatistics.drop_duplicates(subset='teamName', keep='last')
TeamIdMap = IdUnique.set_index('teamName')['teamId']


# Usar .map() para agregar la columna TeamID a df1
mergedDfTeams['teamId'] = mergedDfTeams['Team'].map(TeamIdMap)
mergedDfTeams

,Team,2024/25,2023/24,2022/23,2021/22,2020/21,2019/20,2018/19,2017/18,2016/17,...,2008/09,2007/08,2006/07,2005/06,2004/05,2003/04,2002/03,2001/02,2000/01,teamId
0,Suns,$220708856,$199598596,$186655694,$157814221,$157038429,$120864909,$135516107,$118874973,$116998090,...,$109308179,$107548369,$101949887,$86623329,$73862884,$114470809,$96444346,$99894176,$98236801,1610612756
1,Timberwolves,$204304153,$171379743,$154583256,$158919956,$158838062,$140076911,$152060122,$148875924,$108880975,...,$95402059,$104111617,$103466650,$99874021,$116136117,$121893806,$103942847,$97242068,$86541172,1610612750
2,Celtics,$196051963,$192495669,$189402747,$157908082,$162002554,$144438990,$156265226,$147862224,$121835160,...,$113699674,$112353166,$97521952,$91922105,$106950918,$102933630,$91769470,$83866497,$93812764,1610612738
3,Knicks,$192251829,$169893032,$157970114,$139506473,$124473662,$122940807,$153837075,$135188201,$133733931,...,$138761126,$145045534,$181708756,$204546649,$169869031,$152974005,$163203998,$151780123,$134868788,1610612752
4,Lakers,$191906169,$174924626,$179603738,$190114264,$169806014,$152058818,$133686642,$132268307,$123551292,...,$112345455,$109514294,$119383317,$117784029,$107748764,$109667222,$110286387,$95773977,$107461347,1610612747
5,Nuggets,$185864258,$186298923,$172125732,$159534234,$158177865,$157915026,$147527819,$137933530,$109283609,...,$100679866,$125093110,$101449214,$91251553,$76352236,$67381969,$85763667,$96124258,$95107581,1610612743
6,Bucks,$185323374,$192913480,$193959293,$186027881,$166502403,$150391306,$163314041,$154578435,$125459700,...,$102459760,$94351897,$98322546,$101799984,$95143914,$91036902,$101281964,$99237401,$105367649,1610612749
7,Mavericks,$178518805,$172740566,$187929929,$146505707,$155575488,$148255740,$108418636,$109584150,$133423098,...,$136466588,$153346197,$137084724,$159033650,$152215779,$136028569,$127382002,$102301160,$95263612,1610612742
8,Warriors,$177010942,$215575491,$203984702,$206963957,$208524600,$158539039,$182393116,$176496341,$132419209,...,$96796633,$93713720,$100666627,$92256901,$90996010,$91288107,$83368514,$84158566,$77200694,1610612744
9,Heat,$176102077,$182407172,$160536360,$162860249,$164195796,$159290850,$190971244,$171384276,$131318946,...,$100932646,$112742426,$99743680,$98100857,$98580937,$80979792,$100204748,$94166335,$133893083,1610612748


Limpieza dataframe Players

In [74]:
#Filtramos la data de Players para quedarnos solo con los jugadores que tenemos en común con la tabla salarios
dfPlayers = Players[Players['personId'].isin(MergedDfPlayers['personId'])].reset_index(drop=True)
dfPlayers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   personId     577 non-null    int64  
 1   firstName    577 non-null    object 
 2   lastName     577 non-null    object 
 3   birthdate    348 non-null    object 
 4   school       344 non-null    object 
 5   country      348 non-null    object 
 6   height       346 non-null    float64
 7   bodyWeight   346 non-null    float64
 8   guard        399 non-null    object 
 9   forward      399 non-null    object 
 10  center       399 non-null    object 
 11  draftYear    348 non-null    float64
 12  draftRound   347 non-null    float64
 13  draftNumber  346 non-null    float64
 14  dleague      348 non-null    object 
dtypes: float64(5), int64(1), object(9)
memory usage: 67.7+ KB


Añadimos una columna para tener las posiciones indicadas directamente

In [172]:
# Definir condiciones
conditions = [
    dfPlayers["guard"] == True,
    dfPlayers["forward"] == True,
    dfPlayers["center"] == True
]

# Definir los valores correspondientes
choices = ["Guard", "Forward", "Center"]

# Crear la nueva columna 'Position'
dfPlayers["Position"] = np.select(conditions, choices, default="Unknown")

dfPlayers

,personId,firstName,lastName,birthdate,school,country,height,bodyWeight,guard,forward,center,draftYear,draftRound,draftNumber,dleague,Position
0,1585,Brandon,Williams,1975-02-27,Davidson,USA,78.0,215.0,False,True,False,-1.0,-1.0,-1.0,False,Forward
1,2074,Jabari,Smith,1977-02-12,Louisiana State,USA,83.0,250.0,False,False,True,2000.0,2.0,45.0,False,Center
2,2544,LeBron,James,1984-12-30,St. Vincent-St. Mary HS (OH),USA,81.0,250.0,False,True,False,2003.0,1.0,1.0,False,Forward
3,101108,Chris,Paul,1985-05-06,Wake Forest,USA,72.0,175.0,True,False,False,2005.0,1.0,4.0,False,Guard
4,200768,Kyle,Lowry,1986-03-25,Villanova,USA,72.0,196.0,True,False,False,2006.0,1.0,24.0,False,Guard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,1642484,RayJ,Dennis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
573,1642486,Boo,Buie III,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
574,1642502,Malevy,Leons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
575,1642505,Alex,Ducas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown


Como nos falta la altura y fecha de nacimiento de mas de 200 registros, decidimos filtrar la data para quedarnos solo con los jugadores de los cuales tenemos toda la informacion completa

In [174]:
dfPlayersFiltered = dfPlayers.dropna(subset=['height', 'birthdate']).reset_index(drop=True)
dfPlayersFiltered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   personId     346 non-null    int64  
 1   firstName    346 non-null    object 
 2   lastName     346 non-null    object 
 3   birthdate    346 non-null    object 
 4   school       342 non-null    object 
 5   country      346 non-null    object 
 6   height       346 non-null    float64
 7   bodyWeight   346 non-null    float64
 8   guard        346 non-null    object 
 9   forward      346 non-null    object 
 10  center       346 non-null    object 
 11  draftYear    346 non-null    float64
 12  draftRound   345 non-null    float64
 13  draftNumber  344 non-null    float64
 14  dleague      346 non-null    object 
 15  Position     346 non-null    object 
dtypes: float64(5), int64(1), object(10)
memory usage: 43.4+ KB


In [176]:
dfPlayersFiltered

,personId,firstName,lastName,birthdate,school,country,height,bodyWeight,guard,forward,center,draftYear,draftRound,draftNumber,dleague,Position
0,1585,Brandon,Williams,1975-02-27,Davidson,USA,78.0,215.0,False,True,False,-1.0,-1.0,-1.0,False,Forward
1,2074,Jabari,Smith,1977-02-12,Louisiana State,USA,83.0,250.0,False,False,True,2000.0,2.0,45.0,False,Center
2,2544,LeBron,James,1984-12-30,St. Vincent-St. Mary HS (OH),USA,81.0,250.0,False,True,False,2003.0,1.0,1.0,False,Forward
3,101108,Chris,Paul,1985-05-06,Wake Forest,USA,72.0,175.0,True,False,False,2005.0,1.0,4.0,False,Guard
4,200768,Kyle,Lowry,1986-03-25,Villanova,USA,72.0,196.0,True,False,False,2006.0,1.0,24.0,False,Guard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,1631260,AJ,Green,1999-09-27,Northern Iowa,USA,77.0,190.0,True,False,False,-1.0,-1.0,-1.0,False,Guard
342,1631288,Jamal,Cain,1999-03-20,Oakland,USA,78.0,191.0,False,True,False,-1.0,-1.0,-1.0,False,Forward
343,1631306,Cole,Swider,1999-05-08,Syracuse,USA,81.0,220.0,False,True,False,-1.0,-1.0,-1.0,False,Forward
344,1631311,Lester,Quinones,2000-11-16,Memphis,USA,76.0,208.0,True,False,False,-1.0,-1.0,-1.0,False,Guard


In [78]:
dfPlayers.to_csv('PlayersFinalClean.csv', index = False)
dfPlayersFiltered.to_csv('dfPlayersFilteredClean.csv', index = False)
mergedDfTeams.to_csv('dfTeamSalariesClean.csv', index = False)
MergedDfPlayers.to_csv('dfPlayerSalariesClean.csv', index = False)
PlayerStatistics.to_csv('PlayersStatisticsEnero.csv', index = False)
TeamStatistics.to_csv('TeamStatisticsEnero.csv', index = False)

## Cuando metimos la data en PowerBI para empezar con la analística de gráficos descubrimos que faltaba la data correspondiente a la Temporada 2023-2024 por lo que volvimos a descargar la data de Kaggle y limpiamos nuevamente los archivos de Statistics Team y Players, aplicando todos los pasos previos a las nuevas tablas

In [186]:
TeamStatistics = pd.read_csv('TeamStatisticsDescargaFebrero.csv', low_memory= False)
PlayerStatistics = pd.read_csv('PlayerStatisticsDescargaFebrero.csv')

C:\Users\andre\AppData\Local\Temp\ipykernel_29804\3496046324.py:2: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  PlayerStatistics = pd.read_csv('PlayerStatisticsDescargaFebrero.csv')


In [187]:
TeamStatistics[['Date', 'Time']] = TeamStatistics['gameDate'].str.split(' ', expand=True)
TeamStatistics['Date'] = pd.to_datetime(TeamStatistics['Date'])
TeamStatistics = TeamStatistics[TeamStatistics['Date'].dt.year >= 2000]
TeamStatistics = TeamStatistics.drop(columns=['Date', 'Time'])
TeamStatistics['gameDate'] = pd.to_datetime(TeamStatistics['gameDate'])

In [188]:
PlayerStatistics[['Date', 'Time']] = PlayerStatistics['gameDate'].str.split(' ', expand=True)
PlayerStatistics['Date'] = pd.to_datetime(PlayerStatistics['Date'])
PlayerStatistics = PlayerStatistics[PlayerStatistics['Date'].dt.year >= 2000]
PlayerStatistics = PlayerStatistics.drop(columns=['Date', 'Time'])
PlayerStatistics['gameDate'] = pd.to_datetime(PlayerStatistics['gameDate'])

In [192]:
TeamStatistics.drop(['coachId'], axis=1, inplace=True)
TeamStatistics['win'] = TeamStatistics['win'].astype(int)
TeamStatistics['home'] = TeamStatistics['home'].astype(int)


In [194]:
TeamStatistics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 131174 entries, 0 to 131173
Data columns (total 47 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   gameId                   131174 non-null  int64         
 1   gameDate                 131174 non-null  datetime64[ns]
 2   teamCity                 131174 non-null  object        
 3   teamName                 131174 non-null  object        
 4   teamId                   131174 non-null  int64         
 5   opponentTeamCity         131174 non-null  object        
 6   opponentTeamName         131174 non-null  object        
 7   opponentTeamId           131174 non-null  int64         
 8   home                     131174 non-null  int32         
 9   win                      131174 non-null  int32         
 10  teamScore                131174 non-null  int64         
 11  opponentScore            131174 non-null  int64         
 12  assists              

Tenemos varias columnas de sobra porque tienen desmasiados valores nulos por lo que vamos a eliminarlas

In [199]:
columnas_a_eliminar = ['benchPoints', 'biggestLead', 'biggestScoringRun', 'leadChanges', 
                     'pointsFastBreak', 'pointsFromTurnovers', 'pointsInThePaint', 
                     'pointsSecondChance', 'timesTied', 'timeoutsRemaining', 
                     'seasonWins', 'seasonLosses']
TeamStatistics = TeamStatistics.drop(columns=columnas_a_eliminar)

In [201]:
TeamStatistics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 131174 entries, 0 to 131173
Data columns (total 35 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   gameId                   131174 non-null  int64         
 1   gameDate                 131174 non-null  datetime64[ns]
 2   teamCity                 131174 non-null  object        
 3   teamName                 131174 non-null  object        
 4   teamId                   131174 non-null  int64         
 5   opponentTeamCity         131174 non-null  object        
 6   opponentTeamName         131174 non-null  object        
 7   opponentTeamId           131174 non-null  int64         
 8   home                     131174 non-null  int32         
 9   win                      131174 non-null  int32         
 10  teamScore                131174 non-null  int64         
 11  opponentScore            131174 non-null  int64         
 12  assists              

In [203]:
TeamStatistics['teamName'] = TeamStatistics['teamName'].replace('Bobcats', 'Hornets')
TeamStatistics['opponentTeamName'] = TeamStatistics['opponentTeamName'].replace('Bobcats', 'Hornets')
TeamStatistics['teamName'] = TeamStatistics['teamName'].replace('SuperSonics', 'Thunder')
TeamStatistics['opponentTeamName'] = TeamStatistics['opponentTeamName'].replace('SuperSonics', 'Thunder')
PlayerStatistics['playerteamName'] = PlayerStatistics['playerteamName'].replace('Bobcats', 'Hornets')
PlayerStatistics['opponentteamName'] = PlayerStatistics['opponentteamName'].replace('Bobcats', 'Hornets')
PlayerStatistics['playerteamName'] = PlayerStatistics['playerteamName'].replace('SuperSonics', 'Thunder')
PlayerStatistics['opponentteamName'] = PlayerStatistics['opponentteamName'].replace('SuperSonics', 'Thunder')

In [205]:
Unique1 = TeamStatistics['gameId'].nunique()
Unique2 = PlayerStatistics['gameId'].nunique()

print(f"Cantidad de valores únicos en df1['ID']: {Unique1}")
print(f"Cantidad de valores únicos en df2['ID']: {Unique2}")

Cantidad de valores únicos en df1['ID']: 34232
Cantidad de valores únicos en df2['ID']: 34231


In [207]:
TeamStatistics.head()

,gameId,gameDate,teamCity,teamName,teamId,opponentTeamCity,opponentTeamName,opponentTeamId,home,win,...,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints,numMinutes,q1Points,q2Points,q3Points,q4Points
0,22400627,2025-03-19 21:00:00,Utah,Jazz,1610612762,Washington,Wizards,1610612764,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22400627,2025-03-19 21:00:00,Washington,Wizards,1610612764,Utah,Jazz,1610612762,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22400753,2025-02-09 14:00:00,Houston,Rockets,1610612745,Toronto,Raptors,1610612761,1,1,...,10.0,58.0,19.0,15.0,7.0,240.0,18.0,19.0,21.0,36.0
3,22400753,2025-02-09 14:00:00,Toronto,Raptors,1610612761,Houston,Rockets,1610612745,0,0,...,7.0,48.0,21.0,10.0,-7.0,240.0,22.0,20.0,21.0,24.0
4,22400754,2025-02-09 14:00:00,Milwaukee,Bucks,1610612749,Philadelphia,76ers,1610612755,1,1,...,19.0,53.0,18.0,12.0,8.0,240.0,40.0,25.0,38.0,32.0


Vemos que las primeras dos filas corresponden a un partido de marzo 2025 por lo que las eliminamos del dataframe

In [209]:
TeamStatistics = TeamStatistics.drop([0, 1])

In [211]:
Unique1 = TeamStatistics['gameId'].nunique()
Unique2 = PlayerStatistics['gameId'].nunique()

print(f"Cantidad de valores únicos en df1['ID']: {Unique1}")
print(f"Cantidad de valores únicos en df2['ID']: {Unique2}")

Cantidad de valores únicos en df1['ID']: 34231
Cantidad de valores únicos en df2['ID']: 34231


In [213]:
PlayerStatistics['Player'] = PlayerStatistics['firstName'].str.cat(PlayerStatistics['lastName'], sep=' ')

In [215]:
PlayerStatistics = PlayerStatistics.copy()

# Diccionario con los valores a reemplazar
replacements = {
    'Aaron rdon': 'Aaron Gordon',
    'Eric rdon': 'Eric Gordon',
    'ga Bitadze': 'Goga Bitadze',
    'Nic Claxton': 'Nicolas Claxton',
    'Rudy bert': 'Rudy Gobert',
    'Santi Aldama': 'Santiago Aldama',
    'Svi Mykhailiuk': 'Sviatoslav Mykhailiuk'
}

# Aplicar reemplazos
PlayerStatistics['Player'] = PlayerStatistics['Player'].replace(replacements)

In [217]:
PlayerStatistics.to_csv('PlayersStatisticsFebrero.csv', index = False)
TeamStatistics.to_csv('TeamStatisticsFebrero.csv', index = False)